In [31]:
from __future__ import division
%matplotlib inline
import CameraNetwork
import CameraNetwork.global_settings as gs
from CameraNetwork.utils import extractThumbnails
import cPickle
import cv2
import datetime
import Image
import ImageDraw
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import skimage.io as io
import time
import tqdm
from tqdm import tqdm_notebook
import traceback
plt.rcParams['figure.figsize'] = (8.0, 8.0)

In [2]:
c = CameraNetwork.CLIclient(timeout=60)
proxy_params = CameraNetwork.retrieve_proxy_parameters()
c.start(proxy_params)
time.sleep(3)
print c.client_instance.servers

ERROR:root:Failed retrieving proxy parameters:
Traceback (most recent call last):
  File "i:\amit\office backup\amit\studies\cameranetwork_git\CameraNetwork\internet.py", line 56, in retrieve_proxy_parameters
    response = urllib2.urlopen(req)
  File "C:\Anaconda2\lib\urllib2.py", line 154, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Anaconda2\lib\urllib2.py", line 435, in open
    response = meth(req, response)
  File "C:\Anaconda2\lib\urllib2.py", line 548, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Anaconda2\lib\urllib2.py", line 473, in error
    return self._call_chain(*args)
  File "C:\Anaconda2\lib\urllib2.py", line 407, in _call_chain
    result = func(*args)
  File "C:\Anaconda2\lib\urllib2.py", line 556, in http_error_default
    raise HTTPError(req.get_full_url(), code, msg, hdrs, fp)
HTTPError: HTTP Error 404: Not Found



['109', '109L', '110', '110L', '111', '111L', '112L', '113', '113L', '114', '114L', '115', '115L', '116', '116L', '118', '119', '120']


In [3]:
local_servers = [server_id for server_id in c.servers_list if server_id.endswith("L")]

In [27]:
def get_thumbs(client, server_id):
    days = client[server_id].days()[1][0]["days_list"]
    thumbnails = []
    dfs = []
    for day in days:
        time_ind = datetime.datetime(*[int(i) for i in day.split("_")])
        try:
            reply = c[server_id].thumbnails(time_ind.date(), "60T")
            thumbnails.extend(extractThumbnails(reply[1][0]["thumbnails"]))
            dfs.append(reply[1][0]["thumbs_df"])
        except Exception, e:
            print(server_id, day, type(e))
    
    if dfs:
        df = pd.concat(dfs)
    else:
        df = None

    return server_id, thumbnails, df

In [28]:
t0 = time.time()
replies = [get_thumbs(c, server_id) for server_id in local_servers]
t1 = time.time() - t0

('109L', '2017_03_22', <type 'exceptions.Exception'>)
('109L', '2017_03_23', <type 'exceptions.Exception'>)
('109L', '2017_03_24', <type 'exceptions.Exception'>)
('109L', '2017_03_26', <type 'exceptions.Exception'>)
('109L', '2017_03_27', <type 'exceptions.Exception'>)
('110L', '2017_03_10', <type 'exceptions.Exception'>)
('111L', '2017_02_20', <type 'exceptions.IOError'>)
('111L', '2017_03_23', <type 'exceptions.Exception'>)
('111L', '2017_03_24', <type 'exceptions.Exception'>)
('112L', '2017_03_21', <type 'exceptions.Exception'>)
('112L', '2017_03_22', <type 'exceptions.Exception'>)
('112L', '2017_03_23', <type 'exceptions.Exception'>)
('112L', '2017_03_24', <type 'exceptions.Exception'>)
('112L', '2017_03_26', <type 'exceptions.Exception'>)
('112L', '2017_03_27', <type 'exceptions.Exception'>)
('112L', '2017_04_02', <type 'exceptions.Exception'>)
('113L', '2017_02_19', <type 'exceptions.Exception'>)
('114L', '2017_02_21', <type 'exceptions.Exception'>)
('115L', '2017_02_20', <type '

In [29]:
t1

3562.9219999313354

In [7]:
import cPickle
with open("thumbs_local.pkl", "wb") as f:
    cPickle.dump(replies, f)

In [32]:
new_dfs = {}
for server, thumbnails, df in replies:
    if df is None:
        continue
            
    paths = []
    for i, (ind, row) in enumerate(df.iterrows()):
        thumbs_path = os.path.join("thumbnails", server, os.path.split(row["path"])[-1])
        thumbs_path = thumbs_path[:-3] + ".jpg"
        paths.append(thumbs_path)
        folder, _ = os.path.split(thumbs_path)
        if not os.path.exists(folder):
            os.makedirs(folder)
        io.imsave(thumbs_path, thumbnails[i])
    df["thumbnail"] = paths
    new_dfs[server] = df

C:\Anaconda2\lib\site-packages\skimage\io\_io.py:132: UserWarning: thumbnails\109L\1486961940.0_2017_02_13_04_59_00_2..jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Anaconda2\lib\site-packages\skimage\io\_io.py:132: UserWarning: thumbnails\109L\1488297300.0_2017_02_28_15_55_00_2..jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Anaconda2\lib\site-packages\skimage\io\_io.py:132: UserWarning: thumbnails\109L\1488383700.0_2017_03_01_15_55_00_2..jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Anaconda2\lib\site-packages\skimage\io\_io.py:132: UserWarning: thumbnails\109L\1489582500.0_2017_03_15_12_55_00_2..jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Anaconda2\lib\site-packages\skimage\io\_io.py:132: UserWarning: thumbnails\110L\1488297300.0_2017_02_28_15_55_00_2..jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Anaconda2\lib\site-packages\skimage\i

In [33]:
with open("thumbnails_downloaded_local.pkl", "wb") as f:
    cPickle.dump(new_dfs, f)

In [34]:
with open("thumbnails_downloaded_remote.pkl", "rb") as f:
    remote_dfs = cPickle.load(f)

In [35]:
remote_dfs.update(new_dfs)

In [37]:
with open("thumbnails_downloaded.pkl", "wb") as f:
    cPickle.dump(remote_dfs, f)

In [40]:
c.servers_list

['109',
 '109L',
 '110',
 '110L',
 '111',
 '111L',
 '112',
 '112L',
 '113',
 '113L',
 '114',
 '114L',
 '115',
 '115L',
 '116',
 '116L',
 '117',
 '117L',
 '118L',
 '119',
 '120']